In [ ]:
# --- 1. Instalación de Librerías ---
# Necesitamos instalar 'openpyxl' para que pandas pueda leer archivos .xlsx
!pip install openpyxl

print("Librería 'openpyxl' instalada correctamente.")
print("-" * 50)

Librería 'openpyxl' instalada correctamente.
--------------------------------------------------


In [1]:
# --- 2. Importación de Librerías ---
import pandas as pd
import glob

print("Librerías importadas.")
print("-" * 50)

# --- 3. Ejecución de la Tarea ---

# 1. Definir los nombres de los archivos a cargar
file_list = ["ventas1.xlsx", "ventas2.xlsx", "ventas3.xlsx","ventas4.xlsx"]

# 2. Definir las columnas que nos interesan
columnas_interesantes = [
    "Fecha",
    "Artículo",
    "Unidades",
    "Total Con impuesto",
    "Familia"
]

# 3. Lista para guardar los dataframes de cada archivo
lista_dfs = []

# 4. Bucle para leer cada archivo
print(f"Iniciando la carga de {len(file_list)} archivos...")
try:
    for file in file_list:
        df = pd.read_excel(file)
        print(f"Archivo '{file}' cargado.")

        columnas_actuales_archivo = df.columns
        columnas_para_extraer = [col for col in columnas_interesantes if col in columnas_actuales_archivo]

        columnas_faltantes = [col for col in columnas_interesantes if col not in columnas_actuales_archivo]
        if columnas_faltantes:
            print(f"  ADVERTENCIA: El archivo '{file}' no tiene las columnas: {columnas_faltantes}")

        if not columnas_para_extraer:
            print(f"  ERROR: El archivo '{file}' no tiene ninguna columna de interés. Se omitirá.")
            continue

        df = df[columnas_para_extraer]
        lista_dfs.append(df)

    if not lista_dfs:
        print("Error: No se pudo leer ningún archivo con datos válidos.")
        exit()

    # 5. Unir todos los dataframes en uno solo
    df_combined = pd.concat(lista_dfs, ignore_index=True, join='outer')
    print(f"Archivos combinados. Total de filas: {len(df_combined)}")

    # 6. Procesar la columna de Fecha
    if "Fecha" in df_combined.columns:
        # Asumimos que el formato es DD/MM/YYYY
        df_combined["Fecha"] = pd.to_datetime(df_combined["Fecha"], dayfirst=True, errors='coerce')
        df_combined = df_combined.dropna(subset=["Fecha"])
        print("Columna 'Fecha' convertida a datetime.")

        # 7. Ordenar por fecha
        df_sorted = df_combined.sort_values(by="Fecha")
        print("DataFrame ordenado por 'Fecha'.")

        # ***** PASO 7.5: SOLUCIÓN AL FORMATO 0:00:00 *****
        # Convertimos la columna de datetime (fecha+hora) a solo fecha.
        df_sorted["Fecha"] = df_sorted["Fecha"].dt.date
        print("Formato de columna 'Fecha' ajustado (eliminada la hora 0:00:00).")
        # **************************************************

    else:
        print("ADVERTENCIA: No se encontró la columna 'Fecha'. El archivo no se puede ordenar por fecha.")
        df_sorted = df_combined

    # 8. Definir el nombre del archivo de salida
    output_file = "Datos_Ventas_Consolidados.xlsx"

    # 9. Guardar en un nuevo archivo Excel
    columnas_finales = [col for col in columnas_interesantes if col in df_sorted.columns]
    df_sorted = df_sorted[columnas_finales]

    df_sorted.to_excel(output_file, index=False, engine='openpyxl')

    print(f"\n¡ÉXITO!")
    print(f"Se ha creado el archivo '{output_file}' con {len(df_sorted)} filas.")
    print(f"Contiene las columnas: {df_sorted.columns.to_list()}")
    print("Puedes descargarlo desde el panel de la izquierda.")

except FileNotFoundError as e:
    print(f"Error: No se encontró el archivo '{e.filename}'.")
    print("Por favor, asegúrate de que 'ventas1.xlsx', 'ventas2.xlsx', 'ventas3.xlsx' y 'ventas4.xlsx' están subidos a Colab.")
except Exception as e:
    print(f"Ha ocurrido un error inesperado: {e}")

Librerías importadas.
--------------------------------------------------
Iniciando la carga de 4 archivos...
Archivo 'ventas1.xlsx' cargado.
Archivo 'ventas2.xlsx' cargado.
Archivo 'ventas3.xlsx' cargado.
Archivo 'ventas4.xlsx' cargado.
Archivos combinados. Total de filas: 13178
Columna 'Fecha' convertida a datetime.
DataFrame ordenado por 'Fecha'.
Formato de columna 'Fecha' ajustado (eliminada la hora 0:00:00).

¡ÉXITO!
Se ha creado el archivo 'Datos_Ventas_Consolidados.xlsx' con 13178 filas.
Contiene las columnas: ['Fecha', 'Artículo', 'Unidades', 'Total Con impuesto', 'Familia']
Puedes descargarlo desde el panel de la izquierda.
